In [ ]:
pip install feedparser

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6105 sha256=5dd777e36aa50b96fde190847a63f8bcaa783eb99b484c1d1cf2fc65c0a42c44
  Stored in directory: c:\users\udayan\appdata\local\pip\cache\wheels\3d\4d\ef\37cdccc18d6fd7e0dd7817dcdf9146d4d6789c32a227a28134
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.4.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install pandas

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   --------------------------- ------------ 7.9/11.5 MB 45.9 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 44.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------- 12.6/12.6 MB 68.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Fetching recent papers from Arxiv

In [6]:
import requests
import feedparser
import pandas as pd
from datetime import datetime

In [ ]:
def fetch_papers(category="physics", max_results=100, sort_by="submittedDate"):
    base_url = "http://export.arxiv.org/api/query?"
    query = f"search_query=cat:{category}&start=0&max_results={max_results}&sortBy={sort_by}&sortOrder=descending"

    response = requests.get(base_url + query)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch data: HTTP {response.status_code}")

    feed = feedparser.parse(response.content)
    
    if not feed.entries:
        print(f"No entries found for category '{category}'. Try a different category or adjust filters.")
        return pd.DataFrame()

    papers = []
    for entry in feed.entries:
        
        ## Code to filter out only preprints
        # journal_ref = getattr(entry, "arxiv_journal_ref", None)
        # doi = getattr(entry, "arxiv_doi", None)

        # # Skip if the paper has been published
        # if journal_ref or doi:
        #     continue

        paper = {
            "title": entry.title,
            "authors": ", ".join(author.name for author in entry.authors),
            "summary": entry.summary,
            "published": entry.published,
            "updated": entry.updated,
            "arxiv_url": entry.link,
            "pdf_url": next((link.href for link in entry.links if link.type == "application/pdf"), None),
            "categories": ", ".join(tag["term"] for tag in entry.tags) if "tags" in entry else "",
            "journal_ref": getattr(entry, "arxiv_journal_ref", None),
            "doi": getattr(entry, "arxiv_doi", None)
        }
        papers.append(paper)

    return pd.DataFrame(papers)

In [ ]:
if __name__ == "__main__":
    category = "astro-ph"
    max_results = 100     # Adjust this number as needed

    df = fetch_papers(category=category, max_results=max_results)

    if not df.empty:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        # df.to_csv(f"arxiv_preprints_{category}_{timestamp}.csv", index=False)
        df.to_json(f"arxiv_preprints_{category}_{timestamp}.json", orient="records", indent=2)
        print(f"Fetched {len(df)} preprints. Files saved as arxiv_preprints_{category}_{timestamp}.csv and .json")
    else:
        print("No preprints found.")

Fetched 100 preprints. Files saved as arxiv_preprints_astro-ph_20250519_103836.csv and .json


# Downloading the PDFs

In [28]:
import os
import requests
import json

In [ ]:
def get_arxiv_pdf_url(arxiv_url):
    """Extract the paper ID from the arxiv_url and construct the direct PDF link."""
    try:
        paper_id = arxiv_url.split('/')[-1]
        return f"https://arxiv.org/pdf/{paper_id}.pdf"
    except Exception:
        return None

def download_pdfs(json_file, output_dir="pdfs"):
    os.makedirs(output_dir, exist_ok=True)
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for idx, entry in enumerate(data):
        arxiv_url = entry.get("arxiv_url")
        pdf_url = get_arxiv_pdf_url(arxiv_url)
        title = entry.get("title", f"paper_{idx}").replace(" ", "_").replace("/", "_")

        if pdf_url:
            try: 
                response = requests.get(pdf_url)
                if response.headers.get('Content-Type') == 'application/pdf':
                    pdf_path = os.path.join(output_dir, f"{title}.pdf")
                    with open(pdf_path, 'wb') as pdf_file:
                        pdf_file.write(response.content)
                    print(f"Downloaded: {title}")
                else:
                    print(f"Failed to download (Not a PDF): {title}")
            except Exception as e:
                print(f"Error downloading {title}: {e}")


if __name__ == "__main__":
    download_pdfs("arxiv_preprints_astro-ph_20250519_103836.json")

Downloaded: Bremsstrahlung_emission_from_nuclear_reactions_in_compact_stars
Downloaded: Towards_a_warped_inflationary_brane_scanning
Error downloading Ultraviolet_Spectra_of_Local_Galaxies_and_their_Link_with_the_High-z
__Population: [Errno 22] Invalid argument: 'pdfs\\Ultraviolet_Spectra_of_Local_Galaxies_and_their_Link_with_the_High-z\n__Population.pdf'
Downloaded: Correlated_variability_in_the_blazar_3C_454.3
Error downloading Biases_and_Uncertainties_in_Physical_Parameter_Estimates_of_Lyman_Break
__Galaxies_from_Broad-band_Photometry: [Errno 22] Invalid argument: 'pdfs\\Biases_and_Uncertainties_in_Physical_Parameter_Estimates_of_Lyman_Break\n__Galaxies_from_Broad-band_Photometry.pdf'
Error downloading Dynamics_of_a_Spherical_Accretion_Shock_with_Neutrino_Heating_and
__Alpha-Particle_Recombination: [Errno 22] Invalid argument: 'pdfs\\Dynamics_of_a_Spherical_Accretion_Shock_with_Neutrino_Heating_and\n__Alpha-Particle_Recombination.pdf'
Downloaded: Asymptotically_FRW_black_holes
Downl

# Extracting the Introduction and Conclusion sections from the pdfs

In [35]:
pip install PyMuPDF


   ---------------------------------------- 0.0/16.6 MB ? eta -:--:--
   -------------- ------------------------- 6.0/16.6 MB 41.5 MB/s eta 0:00:01
   -------------------------------------- - 16.0/16.6 MB 46.2 MB/s eta 0:00:01
   ---------------------------------------- 16.6/16.6 MB 41.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import os
import fitz
import re

def extract_sections(text):
    text = re.sub(r'\n+', '\n', text)

    intro_pattern = re.compile(r'(?:^|\n)(\d{0,2}\.?\s*)?(Introduction)\s*\n', re.IGNORECASE)
    section_heading_pattern = re.compile(r'(?:^|\n)(\d{0,2}\.?\s*)?[A-Z][^\n]{0,50}\s*\n')
    concl_pattern = re.compile(r'(?:^|\n)(\d{0,2}\.?\s*)?(Conclusion|Conclusions|Discussion)\s*\n', re.IGNORECASE)
    end_content_pattern = re.compile(r'(?:^|\n)(\d{0,2}\.?\s*)?(References|Acknowledgments?|Appendix|Bibliography)\s*\n', re.IGNORECASE)

    # Extract Introduction
    intro_match = intro_pattern.search(text)
    intro_text = ""
    if intro_match:
        intro_start = intro_match.end()
        next_section_match = section_heading_pattern.search(text, intro_start)
        if next_section_match:
            intro_text = text[intro_start:next_section_match.start()]
        else:
            intro_text = text[intro_start:]

    # Extract Conclusion
    concl_match = concl_pattern.search(text)
    concl_text = ""
    if concl_match:
        concl_start = concl_match.end()
        end_match = end_content_pattern.search(text, concl_start)
        if end_match:
            concl_text = text[concl_start:end_match.start()]
        else:
            concl_text = text[concl_start:]

    return intro_text.strip(), concl_text.strip()


In [52]:
def extract_text(pdf_dir="pdfs", output_dir="texts"):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(pdf_dir):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, filename)
            doc = fitz.open(pdf_path)
            full_text = ""
            for page in doc:
                full_text += page.get_text()
            
            intro_text, concl_text = extract_sections(full_text)

            extracted_text = f"Introduction:\n{intro_text}\n\nConclusion:\n{concl_text}"

            text_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.txt")
            with open(text_path, 'w', encoding='utf-8') as f:
                f.write(extracted_text)
            
            print(f"Processed: {filename}")

In [53]:
if __name__ == "__main__":
    extract_text()

Processed: 3D_Spectroscopic_Study_of_the_Line_Emitting_Regions_of_Mrk_493.pdf
Processed: Abell_851_and_the_Role_of_Starbursts_in_Cluster_Galaxy_Evolution.pdf
Processed: Active_Galactic_Nuclei,_Radio_Jets_and_Acceleration_of_UHECRs.pdf
Processed: Adiabatic_expansion_and_magnetic_fields_in_AGN_jets.pdf
Processed: Angular_Energy_Distribution_of_Collapsar-Jets.pdf
Processed: Anisotropic_distribution_functions_for_spherical_galaxies.pdf
Processed: An_Evolutionary_Considerations_for_V228_from_47_Tuc.pdf
Processed: Asymptotically_FRW_black_holes.pdf
Processed: A_New_Model_For_Vela_Jr._Supernova_Remnant.pdf
Processed: Black_Holes_Admitting_Strong_Resonant_Phenomena.pdf
Processed: Bremsstrahlung_emission_from_nuclear_reactions_in_compact_stars.pdf
Processed: Bulk_viscosity_of_strange_matter_and_r-modes_in_neutron_stars.pdf
Processed: Compressed_sensing_imaging_techniques_for_radio_interferometry.pdf
Processed: Correlated_variability_in_the_blazar_3C_454.3.pdf
Processed: Detecting_Solar_Neutrino

In [ ]:
import os
import json

def prepare_finetuning_data(json_file, text_dir="texts", output_file="finetune_data.jsonl"):
    with open(json_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as fout:
        for idx, entry in enumerate(metadata):
            title = entry.get("title", "No Title")
            summary = entry.get("summary", "").strip()
            cleaned_title = title.replace(" ", "_").replace("/", "_")
            text_path = os.path.join(text_dir, f"{cleaned_title}.txt")

            if not os.path.exists(text_path):
                continue

            with open(text_path, 'r', encoding='utf-8') as f:
                extracted_text = f.read().strip()

            if not extracted_text:
                continue

            prompt = f"Summarize the Introduction and Conclusion of the following paper titled '{title}':\n\n{extracted_text}"

            response = summary if summary else "To be filled."

            sample = {
                "prompt": prompt,
                "response": response
            }

            fout.write(json.dumps(sample) + "\n")

    print(f"Fine-tuning dataset saved to {output_file}")

if __name__ == "__main__":
    prepare_finetuning_data("arxiv_preprints_astro-ph_20250519_103836.json")


Fine-tuning dataset saved to finetune_data.jsonl


# Fine tuning on smaller models 

In [1]:
pip install transformers datasets accelerate bitsandbytes

  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
   ---------------------------------------- 0.0/75.4 MB ? eta -:--:--
   --- ------------------------------------ 6.6/75.4 MB 32.0 MB/s eta 0:00:03
   ------- -------------------------------- 14.4/75.4 MB 34.7 MB/s eta 0:00:02
   ----------- ---------------------------- 22.5/75.4 MB 36.4 MB/s eta 0:00:02
   ------------------ --------------------- 35.4/75.4 MB 43.1 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install peft

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

In [5]:
dataset = load_dataset("json", data_files="finetune_data.jsonl")
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

def tokenize(batch):
    prompt = batch["prompt"]
    response = batch["response"]
    full_text = prompt + "\n\n#### Summary: \n" + response
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=1024)

tokenized_dataset = dataset.map(tokenize, batched=False)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-summary",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01, 
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available()
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    tokenizer= tokenizer,
    data_collator= data_collator
)

trainer.train()

C:\Users\Udayan\AppData\Local\Temp\ipykernel_9956\3398598816.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
c:\Users\Udayan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.805600
20,3.710300
30,3.620700
40,3.368800
50,3.268300
60,3.436500
70,3.317400
80,3.131900
90,3.206600


TrainOutput(global_step=96, training_loss=3.4160989125569663, metrics={'train_runtime': 1964.9505, 'train_samples_per_second': 0.098, 'train_steps_per_second': 0.049, 'total_flos': 100336140288000.0, 'train_loss': 3.4160989125569663, 'epoch': 3.0})

In [ ]:
# Save the model and tokenizer after training  
trainer.save_model("./gpt2-finetuned-summary")
tokenizer.save_pretrained("./gpt2-finetuned-summary") 

('./gpt2-finetuned-summary\\tokenizer_config.json',
 './gpt2-finetuned-summary\\special_tokens_map.json',
 './gpt2-finetuned-summary\\vocab.json',
 './gpt2-finetuned-summary\\merges.txt',
 './gpt2-finetuned-summary\\added_tokens.json',
 './gpt2-finetuned-summary\\tokenizer.json')

In [9]:
from transformers import pipeline

summarizer = pipeline("text-generation", model="./gpt2-finetuned-summary", tokenizer=tokenizer)
prompt = "Summarize the Introduction and Conclusion of the following paper:\n\nIntroduction: ...\n\nConclusion: ..."
print(summarizer(prompt, max_new_tokens=100)[0]["generated_text"])

Device set to use cpu


Summarize the Introduction and Conclusion of the following paper:

Introduction: ...

Conclusion: ...
We present an introduction and conclusion of the next part of the following paper titled 'The Role of Subatomic Radiation in Photonic Field Properties'
Introduction:
We discuss the relationship between the atomic energy and subatomic radiation. In particular the role of atomic radiation in electromagnetic fields for all the known properties such as luminosity, polarization angle, motion, chromaticity, polarisation, and so on, namely
in particular their photonic properties. This paper presents these properties in a simple,
